In [1]:
#Optimizacion Bayesiana de hiperparametros de  rpart
#funciona automaticamente con EXPERIMENTOS
#va generando incrementalmente salidas para kaggle

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

library("data.table")
library("rlist")
library("yaml")

library("rpart")
library("parallel")

#paquetes necesarios para la Bayesian Optimization
library("DiceKriging")
library("mlrMBO")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,560288,30.0,1165498,62.3,1165498,62.3
Vcells,1013370,7.8,8388608,64.0,1650267,12.6


Loading required package: mlr

Loading required package: ParamHelpers

Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: smoof

Loading required package: checkmate


Attaching package: 'checkmate'


The following object is masked from 'package:DiceKriging':

    checkNames




In [2]:
#para poder usarlo en la PC y en la nube
switch ( Sys.info()[['sysname']],
         Windows = { directory.root   <-  "C:/Users/Jonathan/Desktop/MCD - Laboratorio/7.Labo_1" },   #Microsoft Windows
         Darwin  = { directory.root   <-  "~/dm/" },  #Apple MAC
         Linux   = { directory.root   <-  "~/buckets/b1/crudo/" }  #Entorno Google Cloud
       )
#defino la carpeta donde trabajo
setwd( directory.root )

In [18]:
kexperimento  <- NA   #NA si se corre la primera vez, un valor concreto si es para continuar procesando

kscript           <- "04.05 - BO Rpart FE"
karch_generacion  <- "./datasets/paquete_premium_202011_ext.csv"
karch_aplicacion  <- "./datasets/paquete_premium_202101_ext.csv"
kBO_iter    <-  90   #cantidad de iteraciones de la Optimizacion Bayesiana

hs  <- makeParamSet(
          makeNumericParam("cp"       , lower= -1   , upper=    0.1),
          makeIntegerParam("minsplit" , lower=  1L  , upper= 8000L),  #la letra L al final significa ENTERO
          makeIntegerParam("minbucket", lower=  1L  , upper= 2000L),
          makeIntegerParam("maxdepth" , lower=  3L  , upper=   20L),
          forbidden = quote( minbucket > 0.5*minsplit ) )


ksemilla_azar  <- 100001

In [4]:
#------------------------------------------------------------------------------
#Funcion que lleva el registro de los experimentos

get_experimento  <- function()
{
  if( !file.exists( "./maestro.yaml" ) )  cat( file="./maestro.yaml", "experimento: 1000" )

  exp  <- read_yaml( "./maestro.yaml" )
  experimento_actual  <- exp$experimento

  exp$experimento  <- as.integer(exp$experimento + 1)
  Sys.chmod( "./maestro.yaml", mode = "0644", use_umask = TRUE)
  write_yaml( exp, "./maestro.yaml" )
  Sys.chmod( "./maestro.yaml", mode = "0444", use_umask = TRUE) #dejo el archivo readonly

  return( experimento_actual )
}

In [5]:
#------------------------------------------------------------------------------
#graba a un archivo los componentes de lista
#para el primer registro, escribe antes los titulos

loguear  <- function( reg, arch=NA, folder="./work/", ext=".txt", verbose=TRUE )
{
  archivo  <- arch
  if( is.na(arch) )  archivo  <- paste0(  folder, substitute( reg), ext )

  if( !file.exists( archivo ) )  #Escribo los titulos
  {
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  linea  <- paste0( format(Sys.time(), "%Y%m%d %H%M%S"),  "\t",     #la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  #grabo al archivo

  if( verbose )  cat( linea )   #imprimo por pantalla
}

In [6]:
#------------------------------------------------------------------------------
#funcion para particionar, es la que Andres reemplaza con caret

particionar  <- function( data, division, agrupa="", campo="fold", start=1, seed=NA )
{
  if( !is.na( seed)  )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x ) }, division, seq( from=start, length.out=length(division) )  ) )

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
           by= agrupa ]
}
#------------------------------------------------------------------------------

In [7]:
ArbolSimple  <- function( fold_test, data, param )
{
  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .", 
                   data= data[ fold != fold_test, ],
                   xval= 0,
                   control= param )

  #aplico el modelo a los datos de testing, fold==2
  prediccion  <- predict( modelo, data[ fold==fold_test, ], type = "prob")

  prob_baja2  <- prediccion[, "BAJA+2"]

  ganancia_testing  <- sum(  data[ fold==fold_test ][ prob_baja2 >0.025,  ifelse( clase_ternaria=="BAJA+2", 48750, -1250 ) ] )

  return( ganancia_testing )
}

In [21]:
#------------------------------------------------------------------------------

ArbolesCrossValidation  <- function( data, param, qfolds, pagrupa, semilla )
{
  divi  <- rep( 1, qfolds )
  particionar( data, divi, seed=semilla, agrupa=pagrupa )

  ganancias  <- mcmapply( ArbolSimple, 
                          seq(qfolds), # 1 2 3 4 5  
                          MoreArgs= list( data, param), 
                          SIMPLIFY= FALSE,
                          mc.cores= 1 )   #se puede subir a 5 si posee Linux o Mac OS

  data[ , fold := NULL ]
  #devuelvo la primer ganancia y el promedio
  return( mean( unlist( ganancias )) *  qfolds )   #aqui normalizo la ganancia
}

In [9]:
#------------------------------------------------------------------------------
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros se pasan como variables globales

EstimarGanancia  <- function( x )
{
   GLOBAL_iteracion  <<-  GLOBAL_iteracion + 1

   xval_folds  <- 5
   ganancia  <-  ArbolesCrossValidation( dataset, param=x, qfolds= xval_folds, pagrupa="clase_ternaria", semilla=ksemilla_azar )

   #si tengo una ganancia superadora, genero el archivo para Kaggle
   if(  ganancia > GLOBAL_ganancia_max )
   {
     GLOBAL_ganancia_max <<-  ganancia  #asigno la nueva maxima ganancia
    
     modelo  <- rpart("clase_ternaria ~ .",
                      data= dataset,
                      xval= 0,
                      control= x )

     #genero el vector con la prediccion, la probabilidad de ser positivo
     prediccion  <- predict( modelo, dapply)

     prob_baja2  <- prediccion[, "BAJA+2"]
     Predicted   <- ifelse( prob_baja2 > 0.025, 1, 0 )

     entrega  <-  as.data.table( list( "numero_de_cliente"=dapply$numero_de_cliente, "Predicted"=Predicted)  )

     #genero el archivo para Kaggle
     fwrite( entrega, 
             file= paste0(kkaggle, GLOBAL_iteracion, ".csv" ),
             sep=  "," )
   }

   #logueo 
   xx  <- x
   xx$xval_folds  <-  xval_folds
   xx$ganancia  <- ganancia
   loguear( xx,  arch= klog )


   return( ganancia )
}
#------------------------------------------------------------------------------

In [11]:
#Aqui empieza el programa

if( is.na(kexperimento ) )   kexperimento <- get_experimento()  #creo el experimento

[1] 1001

In [12]:
#en estos archivos quedan los resultados
kbayesiana  <- paste0("./work/E",  kexperimento, "_rpart.RDATA" )
klog        <- paste0("./work/E",  kexperimento, "_rpart_log.txt" )
kkaggle     <- paste0("./kaggle/E",kexperimento, "_rpart_kaggle_" )

In [13]:
GLOBAL_ganancia_max  <-  -Inf
GLOBAL_iteracion  <- 0

In [14]:
if( file.exists(klog) )
{
 tabla_log  <- fread( klog)
 GLOBAL_iteracion  <- nrow( tabla_log ) -1
 GLOBAL_ganancia_max  <-  tabla_log[ , max(ganancia) ]
}

In [15]:
#cargo los datasets
dataset  <- fread(karch_generacion)   #donde entreno
dapply  <- fread(karch_aplicacion)    #donde aplico el modelo

In [19]:
#Aqui comienza la configuracion de la Bayesian Optimization

configureMlr( show.learner.output = FALSE)

funcion_optimizar  <- EstimarGanancia

#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar,
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,
              has.simple.signature = FALSE
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 600,  save.file.path= kbayesiana)
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km  <-  makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))

In [22]:
#inicio la optimizacion bayesiana
if(!file.exists(kbayesiana)) {
  run  <- mbo(obj.fun, learner = surr.km, control = ctrl)
} else  run  <- mboContinue( kbayesiana )   #retomo en caso que ya exista

Computing y column(s) for design. Not provided.



20210924 170921	-0.0976199316675774	1235	381	10	5	8212500
20210924 171146	-0.223040826257784	4764	355	13	5	7515000
20210924 171348	-0.726423298436566	5895	1276	20	5	6757500
20210924 171538	-0.547949130124471	3773	715	8	5	7460000
20210924 171738	-0.29003388373967	7122	1169	12	5	7371250
20210924 172035	-0.14742603412742	6077	21	17	5	6932500
20210924 172231	-0.601270701251633	5210	1914	14	5	7200000
20210924 172333	-0.488527758760029	7643	955	4	5	7488750
20210924 172534	-0.912083183937648	3087	593	10	5	7692500
20210924 172634	0.0585211833007635	2187	248	9	5	0
20210924 172746	-0.946035111416131	6595	1784	5	5	6751250
20210924 172957	-0.341571937302069	4478	1073	19	5	7237500
20210924 173211	-0.267447549273493	6772	702	20	5	7342500
20210924 173346	-0.0882436665066052	3398	192	7	5	7441250
20210924 173508	-0.643912304326659	5388	1841	6	5	6893750
20210924 173830	-0.526583998673595	1213	6	17	5	6913750


[mbo] 0: cp=-0.0976; minsplit=1235; minbucket=381; maxdepth=10 : y = 8.21e+06 : 172.0 secs : initdesign

[mbo] 0: cp=-0.223; minsplit=4764; minbucket=355; maxdepth=13 : y = 7.52e+06 : 144.8 secs : initdesign

[mbo] 0: cp=-0.726; minsplit=5895; minbucket=1276; maxdepth=20 : y = 6.76e+06 : 122.4 secs : initdesign

[mbo] 0: cp=-0.548; minsplit=3773; minbucket=715; maxdepth=8 : y = 7.46e+06 : 109.9 secs : initdesign

[mbo] 0: cp=-0.29; minsplit=7122; minbucket=1169; maxdepth=12 : y = 7.37e+06 : 119.5 secs : initdesign

[mbo] 0: cp=-0.147; minsplit=6077; minbucket=21; maxdepth=17 : y = 6.93e+06 : 177.5 secs : initdesign

[mbo] 0: cp=-0.601; minsplit=5210; minbucket=1914; maxdepth=14 : y = 7.2e+06 : 116.1 secs : initdesign

[mbo] 0: cp=-0.489; minsplit=7643; minbucket=955; maxdepth=4 : y = 7.49e+06 : 61.7 secs : initdesign

[mbo] 0: cp=-0.912; minsplit=3087; minbucket=593; maxdepth=10 : y = 7.69e+06 : 120.9 secs : initdesign

[mbo] 0: cp=0.0585; minsplit=2187; minbucket=248; maxdepth=9 : y =

20210924 173919	0.0998450988050043	7617	1968	10	5	0


[mbo] 1: cp=0.0998; minsplit=7617; minbucket=1968; maxdepth=10 : y = 0 : 45.9 secs : infill_ei



...

[mbo] 88: cp=-0.786; minsplit=7992; minbucket=1; maxdepth=16 : y = 7.05e+06 : 280.4 secs : infill_ei



20210924 220328	-0.0275356450238252	3994	1997	20	5	7170000


[mbo] 89: cp=-0.0275; minsplit=3994; minbucket=1997; maxdepth=20 : y = 7.17e+06 : 178.9 secs : infill_ei

Saved the current state after iteration 90 in the file ./work/E1001_rpart.RDATA.



20210924 220438	0.0999857439952051	1691	837	20	5	0


[mbo] 90: cp=0.1; minsplit=1691; minbucket=837; maxdepth=20 : y = 0 : 65.9 secs : infill_ei

Saved the final state in the file ./work/E1001_rpart.RDATA



In [23]:
quit( save="no" )